## Project 2: Thermoelectric Generator

Import and Install Libraries

In [14]:
%pip install pandas pyfluids sympy scipy numpy matplotlib

import pandas as pd # This makes nice tables
import pyfluids # This does the thermodynamics
import math
import scipy.optimize
import numpy as np
import matplotlib.pyplot as plt


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Power Requirements

In [15]:
power_data = [
    ["Raspberry Pi Pico", 42, 0.1386],
    ["Lora Radio Module", 125, 0.4125],
    ["pH Sensor", 13.3, 0.04389],
    ["Salinity", 34, 0.1122],
]
power_draw = pd.DataFrame(power_data, columns=["Component", "Peak Amperage Draw (mA)", "Power Draw (W)"])

power_draw

,Component,Peak Amperage Draw (mA),Power Draw (W)
0,Raspberry Pi Pico,42.0,0.13860
1,Lora Radio Module,125.0,0.41250
2,pH Sensor,13.3,0.04389
3,Salinity,34.0,0.11220


Temperature Data

In [16]:
minT_data = [
    [1,24],
    [2,25],
    [3,26],
    [4,21],
]
min_Temp = pd.DataFrame(minT_data, columns=["Month", "Min Average Temp (C)"])

min_Temp

,Month,Min Average Temp (C)
0,1,24
1,2,25
2,3,26
3,4,21


In [17]:
maxT_data = [
    [1,32],
    [2,36],
    [3,37],
    [4,32],
]
max_Temp = pd.DataFrame(maxT_data, columns=["Month", "Max Average Temp (C)"])

max_Temp

,Month,Max Average Temp (C)
0,1,32
1,2,36
2,3,37
3,4,32


Solar Irradiation

In [18]:
irradiation_data = [
    ["Min",4.38],
    ["Max",4.93],
]
irradiation = pd.DataFrame(irradiation_data, columns=["Solar Irradiation", "Daily Output (kWh/m^2)"])

irradiation

,Solar Irradiation,Daily Output (kWh/m^2)
0,Min,4.38
1,Max,4.93


Wind Speed

In [27]:
wind_Data = [[3]]
wind_Speed = pd.DataFrame(wind_Data,columns = ["Average Wind Speed (m/s)"])
wind_Speed

,Average Wind Speed (m/s)
0,3


Heat Transfer Characteristics

In [19]:
heat_data = [
    ["Number of Units",21,"3 sets of 7 in series"],
    ["Unit Length",0.03,"m"],
    ["Unit Thickness",3.75E-3,"m"],
    ["Thermal Conductivity",1.72,"W/mK"],
    ["Emissivity",.9,"Painted Black"],
    ["Absorptivity",.9,"Painted Black"],
    ["Assumed Water Temperature",30,"Degree C"],
]
heat_parameters = pd.DataFrame(heat_data, columns=["Parameter", "Value","Units/Notes"])

heat_parameters

,Parameter,Value,Units/Notes
0,Number of Units,21.00000,3 sets of 7 in series
1,Unit Length,0.03000,m
2,Unit Thickness,0.00375,m
3,Thermal Conductivity,1.72000,W/mK
4,Emissivity,0.90000,Painted Black
5,Absorptivity,0.90000,Painted Black
6,Assumed Water Temperature,30.00000,Degree C


First Equation

In [20]:
def topThermalEq(alpha,G,sigma,epsilon,h,Th,Tamb,Tc,Kt,t):
    RHS = G*alpha - sigma * epsilon * (Th)**4 - h * (Th-Tamb) 
    LHS = Kt * t * (Th-Tc)
    

Natural Convection for the Bottom of the Plate

In [21]:
def calcBeta(rho,rhoAmb,Tplate,Tw):
    Beta = -1/rho * (rhoAmb-rho)/(Tw-Tplate)
    return Beta

In [22]:
def calcRalyeighNumber(Beta,Tplate, Tw, x, alpha):
    #Find dT
    deltaT = Tplate - Tw
    #Define Constants
    g = 9.81
    kinematic_viscosity = 8.005E-7
    #Calculate Ra
    Rax  = abs(g * Beta * deltaT * (x)**3 / (kinematic_viscosity * alpha))
    return Rax

In [23]:
def nusseltNumber(rho,rhoAmb,Tplate,Tw,L,Pr,alpha):

    beta = calcBeta(rho,rhoAmb,Tplate,Tw)
    
    #Calculate Ralyeigh Number at x = L
    RaL = calcRalyeighNumber(beta,Tplate, Tw, L, alpha)

    #Critical Rayleigh Number
    if RaL < 1E9:
        NuL = 0.68 + 0.670 * (RaL) ** (1/4) / (( 1- (0.492/Pr) ** (9/16)) ** (4/9))
    else:
        NuL = (0.825+ 0.397 * (RaL) ** (1/6) / (( 1 + (0.492/Pr) ** (9/16)) ** (8/27))**2)



Forced Convection on Top of the Plate

In [25]:
def topForcedConvectionCoefficient(u,L, h, kf):
    kinematic_viscosity = 1.61E-5
    Re = (u ** 2) * L / kinematic_viscosity
    Pr = 1
    NuL = 0.453 * (Re ** (1/2)) * (Pr ** 1/3)
    h = NuL * kf / L
    
    return h
